## **Dependencies**

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

## **Load JSON**

Choose respective json file to transform to data frame
Json be found in /src/response_[ORG]

### Define Params

In [2]:
##########
# PARAMS #
##########

abbreviation = "aa" # abbreviation of development bank
response_folder = "../../../src/responses/"
transformed_folder = "../../../src/transformed/"
response_file = response_folder + f"response_{abbreviation}.json" # json format -> fetched in iati.ipynb
output_file = transformed_folder + f"transformed_{abbreviation}.csv"

In [3]:
print(response_file)

../../../src/responses/response_aa.json


In [4]:
#Read in data into df from json
df = pd.read_json(response_file)
df.head(10)

,sector_code,iati_identifier,title_narrative,reporting_org_ref,sector_vocabulary,activity_date_type,activity_status_code,description_narrative,last_updated_datetime,recipient_region_code,activity_date_iso_date,location_name_narrative,reporting_org_narrative,title_narrative_xml_lang,description_narrative_xml_lang,contact_info_department_narrative,recipient_country_code
0,[15240],XM-DAC-5-7-6612417,[Building Capacity of States in Southeast Asia...,XM-DAC-5-7,[1],"[1, 3]",4,[strengthen national capacities to effectively...,2021-08-18T08:55:11Z,[798],"[2019-01-01T00:00:00Z, 2019-12-31T00:00:00Z]",[Reg. Asien (alle Länder)],"[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[OR 10],NaN
1,[15152],XM-DAC-5-7-6612437,[Supporting a people-oriented Parliament in My...,XM-DAC-5-7,[1],"[1, 3]",4,[The people of Myanmar have improved knowledge...,2021-08-17T21:33:37Z,NaN,"[2019-03-01T00:00:00Z, 2020-12-31T00:00:00Z]","[Rangun, Myanmar]","[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[S 03],[MM]
2,[41081],XM-DAC-5-7-6612446,[German Participation at National Science and ...,XM-DAC-5-7,[1],"[1, 3]",4,[On the basis of descriptive examples at the b...,2021-08-18T08:42:47Z,NaN,"[2019-05-01T00:00:00Z, 2019-09-30T00:00:00Z]","[Bangkok, Thailand]","[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[Bangkok (B)],[TH]
3,[11120],XM-DAC-5-7-90025547,[10 years initiative Schools - Partners for th...,XM-DAC-5-7,[1],"[1, 3]",3,[The PASCH initiative globally conntects pupil...,2019-06-18T13:24:20Z,NaN,"[2018-04-01T00:00:00Z, 2018-12-31T00:00:00Z]",[Reg. Östl. Asien (nur ODA-Länder)],"[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[610],"[CN, VN]"
4,[11120],XM-DAC-5-7-90024987,[10 years initiative Schools - Partners for th...,XM-DAC-5-7,[1],"[1, 3]",4,[The PASCH initiative globally conntects pupil...,2019-06-13T15:10:54Z,NaN,"[2018-04-24T00:00:00Z, 2018-12-10T00:00:00Z]",[Reg. Süd-Zentral Asien (nur ODA-Länder)],"[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[610],"[GE, TJ, UZ]"
5,[11120],XM-DAC-5-7-90025288,[10 years initiative Schools - Partners for th...,XM-DAC-5-7,[1],"[1, 3]",4,[The PASCH initiative globally conntects pupil...,2019-06-18T13:24:22Z,NaN,"[2018-09-15T00:00:00Z, 2018-11-30T00:00:00Z]",[Sri Lanka],"[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[610],[LK]
6,[41010],XM-DAC-5-7-6612469,"[Living Heritage Conference, Living Heritage C...",XM-DAC-5-7,[1],"[1, 3]",4,[Intensification of the local awareness of the...,2020-06-19T08:31:52Z,NaN,"[2019-02-15T00:00:00Z, 2019-03-27T00:00:00Z]","[Colombo, Sri Lanka]","[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[Colombo (B)],[LK]
7,[15110],XM-DAC-5-7-6611363,"[Seminar Diplomats from Laos/Cambodia, Lehrgan...",XM-DAC-5-7,[1],"[1, 3]",4,"[Capacity building in the field of diplomacy, ...",2019-06-13T15:10:55Z,NaN,"[2018-05-24T00:00:00Z, 2018-06-12T00:00:00Z]",[Deutschland],"[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[1-DA],"[LA, KH]"
8,[16010],XM-DAC-5-7-6612524,[Purchase of furniture for an old home in Quet...,XM-DAC-5-7,[1],"[1, 3]",4,[The project should provide shelter for old pe...,2019-06-13T15:10:56Z,NaN,"[2018-09-01T00:00:00Z, 2018-12-15T00:00:00Z]","[Quetta, Pakistan]","[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[Karachi (GK)],[PK]
9,[15160],XM-DAC-5-7-6612529,[Human Rights Defenders and Victims' Families ...,XM-DAC-5-7,[1],"[1, 3]",4,[Training of human rights defenders and victim...,2021-08-18T08:56:30Z,NaN,"[2019-03-01T00:00:00Z, 2019-12-31T00:00:00Z]",[Philippinen],"[Germany - Federal Foreign Office, Deutschland...","[en, de]","[en, de]",[OR 06],[PH]


In [5]:
print(f"Entries: {len(df)}")
df.reporting_org_ref.value_counts()

Entries: 7299


XM-DAC-5-7    7299
Name: reporting_org_ref, dtype: int64

In [6]:
# create new empty df to fill with transformed data

trans_df = pd.DataFrame()

## **Feature Transformation & Engineering**

### IATI ID

In [7]:
trans_df["iati_id"] = df["iati_identifier"].values

trans_df.head(2)

,iati_id
0,XM-DAC-5-7-6612417
1,XM-DAC-5-7-6612437


### Title

In [8]:
############
# EN title #
############

pbar = tqdm(total=len(df))
trans_df["title_en"] = "NaN"

for index, row in df.iterrows():
    title_row = row['title_narrative']
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # nan in pandas is type float
            # check if nan and if yes take first entry in lang
            if isinstance(lang_list, float):
                if isinstance(title_row, float):
                    trans_df["title_en"][index] = "NaN"
                else:
                    trans_df["title_en"][index] = title_row[0]
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if "en" or "EN" in lang_list:
                        if lang_list[j].lower() == "en":
                            title = title_row[j]
                            trans_df["title_en"][index] = title
                    else:
                        trans_df["title_en"][index] = "NaN"
            else:
                pass
        else:
            trans_df["title_en"][index] = row['title_narrative'][0]
    except:
        print(f"Error: Index: {index}, Row: {lang_list}, {title_row}")
    
    pbar.update(1)

pbar.close()

trans_df.head(2)

  0%|          | 0/7299 [00:00<?, ?it/s]

,iati_id,title_en
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...


In [9]:
###################
# Add other title #
###################

pbar = tqdm(total=len(df))

trans_df["title_other"] = "NaN"

for index, row in df.iterrows():
    title_row = row['title_narrative']
    try:
        if 'title_narrative_xml_lang' in df.columns:
            lang_list = row['title_narrative_xml_lang']

            # every title which is has no lang attribute is classified as English and therefore not in other
            if isinstance(lang_list, float):
                trans_df["title_other"][index] = "NaN"
            elif len(lang_list) == len(title_row):
                for j in range(0, len(lang_list)):
                    if lang_list[j].lower() != "en":
                        title = row['title_narrative'][j]
                        if trans_df["title_other"][index] == "NaN":
                            trans_df["title_other"][index] = title
                        else:
                            trans_df["title_other"][index] = f"{trans_df['title_other'][index]}; {title}"
            else:
                trans_df["title_other"][index] = title
            
            pbar.update(1)

        else:
            trans_df["title_other"][index] = "NaN"
        
    except Exception as e:
        print(f"Error: Index: {index} \n Row: {row} \n Exception: {e}")

pbar.close()

trans_df.head(200)

  0%|          | 0/7299 [00:00<?, ?it/s]

,iati_id,title_en,title_other
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...
2,XM-DAC-5-7-6612446,German Participation at National Science and T...,Deutsche Beteiligung an der National Science a...
3,XM-DAC-5-7-90025547,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...
4,XM-DAC-5-7-90024987,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...
...,...,...,...
195,XM-DAC-5-7-6615087,Furnishings for the centre for disabled in Tbi...,Einrichtungsgegenstände für das Behindertenzen...
196,XM-DAC-5-7-6615097,"Ensuring sustainable, safe, and reliable urban...","Gewährleistung von nachhaltiger, sicherer und ..."
197,XM-DAC-5-7-6615098,Membership contribution SEAMEO 2020,Mitgliedsbeitrag SEAMEO 2020
198,XM-DAC-5-7-6615103,Humanitarian Response to COVID-19 in district ...,Balochistan Rural Support Programme (BRSP) - H...


### Organization

In [10]:
trans_df['organization'] = df['reporting_org_narrative'].apply(lambda x: x[0])

print(trans_df.organization.value_counts())
trans_df.head(2)

Germany - Federal Foreign Office    7299
Name: organization, dtype: int64


,iati_id,title_en,title_other,organization
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office


### Country

In [11]:
trans_df["country"] = df["recipient_country_code"]
trans_df.head(5)

,iati_id,title_en,title_other,organization,country
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM]
2,XM-DAC-5-7-6612446,German Participation at National Science and T...,Deutsche Beteiligung an der National Science a...,Germany - Federal Foreign Office,[TH]
3,XM-DAC-5-7-90025547,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[CN, VN]"
4,XM-DAC-5-7-90024987,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[GE, TJ, UZ]"


### Region

In [12]:
trans_df['region'] = df['recipient_region_code']
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798]
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN
2,XM-DAC-5-7-6612446,German Participation at National Science and T...,Deutsche Beteiligung an der National Science a...,Germany - Federal Foreign Office,[TH],NaN
3,XM-DAC-5-7-90025547,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[CN, VN]",NaN
4,XM-DAC-5-7-90024987,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[GE, TJ, UZ]",NaN


### Location

In [13]:
try: 
    if 'title_narrative_xml_lang' in df.columns:
        trans_df['location'] = df['location_name_narrative']
    else:
        trans_df['location'] = "NaN"
except:
        trans_df['location'] = "NaN"    
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798],[Reg. Asien (alle Länder)]
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN,"[Rangun, Myanmar]"
2,XM-DAC-5-7-6612446,German Participation at National Science and T...,Deutsche Beteiligung an der National Science a...,Germany - Federal Foreign Office,[TH],NaN,"[Bangkok, Thailand]"
3,XM-DAC-5-7-90025547,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[CN, VN]",NaN,[Reg. Östl. Asien (nur ODA-Länder)]
4,XM-DAC-5-7-90024987,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[GE, TJ, UZ]",NaN,[Reg. Süd-Zentral Asien (nur ODA-Länder)]


### Descriptions

descr1, descri2, ... can have overlap. Doppelte Beschriebungen entfernen! Es gibt lib um overlap heruaszufinden!

In [14]:
#################################
# Description english and other #
#################################

trans_df["description_en"] = "NaN"
trans_df["description_other"] = "NaN"

for index, row in df.iterrows():

    try:
        if 'description_narrative_xml_lang' in df.columns:
            descr_list = row['description_narrative_xml_lang']
            descr_row = row['description_narrative']

            # nan in pandas is type float
            # check if nan and if yes take first entry in descr
            if isinstance(descr_list, float):
                if isinstance(descr_row, float):
                    trans_df["description_en"][index] = "NaN"
                else:
                    trans_df["description_en"][index] = descr_row[0]
            else:
                # iterate throug description list
                for j in range(0, len(descr_list)):
                    # if description english
                    if descr_list[j].lower() == "en":
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_en"][index] == "NaN":
                            trans_df["description_en"][index] = descr
                        else:
                            trans_df["description_en"][index] = f"{trans_df['description_en'][index]}; {descr}"
                    else:
                        if type(descr_row) == float:
                            descr = "NaN"
                        else:
                            descr = descr_row[j]
                        if trans_df["description_other"][index] == "NaN":
                            trans_df["description_other"][index] = descr
                        else:
                            trans_df["description_other"][index] = f"{trans_df['description_other'][index]}; {descr}"
        else:
            descr_str = ""
            for d in row['description_narrative']:
                descr_str += f"{d}; "
            trans_df["description_en"][index] = descr_str

    except:
        print(f"Error: Index: {index} \n Row: {row}")
        
trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798],[Reg. Asien (alle Länder)],strengthen national capacities to effectively ...,Förderung des Dialogs zu Themen des konvention...
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN,"[Rangun, Myanmar]",The people of Myanmar have improved knowledge ...,Die Bevölkerung von Myanmar weiß besser über d...
2,XM-DAC-5-7-6612446,German Participation at National Science and T...,Deutsche Beteiligung an der National Science a...,Germany - Federal Foreign Office,[TH],NaN,"[Bangkok, Thailand]",On the basis of descriptive examples at the bo...,Anhand von anschaulichen Beispielen auf dem Me...
3,XM-DAC-5-7-90025547,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[CN, VN]",NaN,[Reg. Östl. Asien (nur ODA-Länder)],The PASCH initiative globally conntects pupils...,Die PASCH-Initiative verbindet über Deutsch al...
4,XM-DAC-5-7-90024987,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[GE, TJ, UZ]",NaN,[Reg. Süd-Zentral Asien (nur ODA-Länder)],The PASCH initiative globally conntects pupils...,Die PASCH-Initiative verbindet über Deutsch al...


### Status

In [15]:
# https://iatistandard.org/en/iati-standard/203/codelists/activitystatus/
activity_status = {
    1: "Pipeline/identification",
    2: "Implementation",
    3: "Finalisation",
    4: "Closed",
    5: "Cancelled",
    6: "Suspended"
}

trans_df["status"] = df.activity_status_code
trans_df['status'] = trans_df['status'].replace(activity_status)

trans_df.head(2)
    

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798],[Reg. Asien (alle Länder)],strengthen national capacities to effectively ...,Förderung des Dialogs zu Themen des konvention...,Closed
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN,"[Rangun, Myanmar]",The people of Myanmar have improved knowledge ...,Die Bevölkerung von Myanmar weiß besser über d...,Closed


### Date

In [16]:
#############
# Date Type #
#############

# One Hot
# 1 -> Yes
# 0 -> No

# Codes:
# 1 Planned start
# 2 Actual start
# 3 Planned end
# 4 Actual end

trans_df["planned_start"] = "NaN"
trans_df["actual_start"] = "NaN"
trans_df["planned_end"] = "NaN"
trans_df["actual_end"] = "NaN"

date_types = {
    1: "planned_start",
    2: "actual_start",
    3: "planned_end",
    4: "actual_end"
}

for index, row in df.iterrows():
    dtype_list = row["activity_date_type"]
    iso_date_list = row["activity_date_iso_date"]

    combined_list = list(zip(dtype_list, iso_date_list))

    # replace nums with column names from date_types
    combined_list = [(date_types[int(t[0])], t[1]) for t in combined_list]

    for i in combined_list:
        trans_df[i[0]] = i[1]

trans_df.head(1800)


,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798],[Reg. Asien (alle Länder)],strengthen national capacities to effectively ...,Förderung des Dialogs zu Themen des konvention...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN,"[Rangun, Myanmar]",The people of Myanmar have improved knowledge ...,Die Bevölkerung von Myanmar weiß besser über d...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN
2,XM-DAC-5-7-6612446,German Participation at National Science and T...,Deutsche Beteiligung an der National Science a...,Germany - Federal Foreign Office,[TH],NaN,"[Bangkok, Thailand]",On the basis of descriptive examples at the bo...,Anhand von anschaulichen Beispielen auf dem Me...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN
3,XM-DAC-5-7-90025547,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[CN, VN]",NaN,[Reg. Östl. Asien (nur ODA-Länder)],The PASCH initiative globally conntects pupils...,Die PASCH-Initiative verbindet über Deutsch al...,Finalisation,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN
4,XM-DAC-5-7-90024987,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[GE, TJ, UZ]",NaN,[Reg. Süd-Zentral Asien (nur ODA-Länder)],The PASCH initiative globally conntects pupils...,Die PASCH-Initiative verbindet über Deutsch al...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,XM-DAC-5-7-6614532,"Water collection systems for Nuevo San Carlos,...",Regenwasserauffangsysteme für Nuevo San Carlos...,Germany - Federal Foreign Office,[GT],NaN,[Guatemala],The project consists of providing 90 families ...,"Das Projekt besteht darin, 90 Familien in drei...",Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN
1796,XM-DAC-5-7-6614533,Construction of sanitary facilities for two sc...,Konstruktion von sanitären Anlagen für zwei Sc...,Germany - Federal Foreign Office,[GT],NaN,[Guatemala],The project consists of improving basic sanita...,Das Projekt besteht in der Verbesserung der sa...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN
1797,XM-DAC-5-7-6614542,15th Seminar for Diplomats from Latinamerica/C...,15. Lehrgang für Diplomaten Lateinamerika/Karibik,Germany - Federal Foreign Office,NaN,[498],[Deutschland],Capacity building in the field of diplomacy,Kapazitätsaufbau im diplomatischen Dienst,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN
1798,XM-DAC-5-7-6614562,COVID-19 Support for an indigenous women's org...,COVID-19 Unterstützung einer indigenen Fraueno...,Germany - Federal Foreign Office,[BO],NaN,"[La Paz, Bolivien]",The project focuses on promoting the generatio...,Das Projekt konzentriert sich auf die Förderun...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN


### Last Update

In [17]:
trans_df['last_update'] = df['last_updated_datetime']
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798],[Reg. Asien (alle Länder)],strengthen national capacities to effectively ...,Förderung des Dialogs zu Themen des konvention...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-18T08:55:11Z
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN,"[Rangun, Myanmar]",The people of Myanmar have improved knowledge ...,Die Bevölkerung von Myanmar weiß besser über d...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-17T21:33:37Z


### Sector Codes (CRS & SGDs)

In [18]:
sector_codes = {
    1: "OECD DAC CRS Purpose Codes (5 digit)",
    2: "OECD DAC CRS Purpose Codes (3 digit)",
    3: "Classification of the Functions of Government (UN)",
    4: "Statistical classification of economic activities in the European Community",
    5: "National Taxonomy for Exempt Entities (USA)",
    6: "AidData",
    7: "SDG Goal",
    8: "SDG Target",
    9: "SDG Indicator",
    10: "Humanitarian Global Clusters (Inter-Agency Standing Committee)",
    11: "North American Industry Classification System (NAICS)",
    12: "UN System Function",
    99: "Reporting Organisation", # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity
    98: "Reporting Organisation 2" # The sector reported corresponds to a sector vocabulary maintained by the reporting organisation for this activity (if they are referencing more than one)
}

In [20]:
crs5_df = pd.read_csv("../../../src/codelists/crs5_codes.csv")
crs5_df.head(2)

,code,name,description,language,category,category-name,category-description
0,11110,Education policy and administrative management,"Education sector policy, planning and programm...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...
1,11120,Education facilities and training,"Educational buildings, equipment, materials; s...",en,111,"Education, Level Unspecified",The codes in this category are to be used only...


In [21]:
crs3_df = pd.read_csv("../../../src/codelists/crs3_codes.csv")
crs3_df.head(2)

,code,name,description,language,category,category-name,category-description
0,111,"Education, Level Unspecified",The codes in this category are to be used only...,en,NaN,NaN,NaN
1,112,Basic Education,NaN,en,NaN,NaN,NaN


#### Get CRS5 & derive crs3 tags from crs5 tags

- other tags are not given by organizations in IATI data; need other ways to find out these tags


- It is possible to derive CRS3 tags from CRS5 tags by looking at the first 3 characters.

In [22]:
# to process different variants of codes (crs3, crs5 etc.)

def process_codes(combined_list, translation_df, code_index):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"
    
def derive_crs3(combined_list, translation_df, code_index="1"):
    code_text = ""
    codes_nums = ""
    if any(item[0] == code_index for item in combined_list):
        for i in combined_list:
            if i[0] == code_index:
                translation = translation_df.loc[translation_df['code'] == int(i[1][:3]), 'name'].values[0]
                code_text += f"{translation}; "
                codes_nums += f"{i[1][:3]}; "
        return code_text, codes_nums
    else:
        return "NaN", "NaN"

In [23]:
###############
# ECTRACT CRS #
###############
trans_df["crs_5_code"] = "NaN"
trans_df["crs_5_name"] = "NaN"

# Most Project dont have information on crs3 -> crs3 derived from crs5 tags
trans_df["crs_3_code"] = "NaN"
trans_df["crs_3_name"] = "NaN"


for index, row in df.iterrows():
    crs_voc_list = row['sector_vocabulary']
    crs_code_list = row['sector_code']

    if type(crs_voc_list) == float:
        pass
    else:
        try:
            combined_list = list(zip(crs_voc_list, crs_code_list))

            # CRS 5
            crs5_str, crs5_codes = process_codes(combined_list, crs5_df, "1")
            trans_df["crs_5_code"][index] = crs5_codes
            trans_df["crs_5_name"][index] = crs5_str
            
            # CRS 3
            crs3_str, crs3_codes = derive_crs3(combined_list, crs3_df)
            trans_df["crs_3_code"][index] = crs3_codes
            trans_df["crs_3_name"][index] = crs3_str

        except:
            print(f"Error on Index {index}, {crs_code_list}")
            pass

trans_df.head(5)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798],[Reg. Asien (alle Länder)],strengthen national capacities to effectively ...,Förderung des Dialogs zu Themen des konvention...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-18T08:55:11Z,15240;,Reintegration and SALW control;,152;,"Conflict, Peace & Security;"
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN,"[Rangun, Myanmar]",The people of Myanmar have improved knowledge ...,Die Bevölkerung von Myanmar weiß besser über d...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-17T21:33:37Z,15152;,Legislatures and political parties;,151;,Government & Civil Society-general;
2,XM-DAC-5-7-6612446,German Participation at National Science and T...,Deutsche Beteiligung an der National Science a...,Germany - Federal Foreign Office,[TH],NaN,"[Bangkok, Thailand]",On the basis of descriptive examples at the bo...,Anhand von anschaulichen Beispielen auf dem Me...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-18T08:42:47Z,41081;,Environmental education/training;,410;,General Environment Protection;
3,XM-DAC-5-7-90025547,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[CN, VN]",NaN,[Reg. Östl. Asien (nur ODA-Länder)],The PASCH initiative globally conntects pupils...,Die PASCH-Initiative verbindet über Deutsch al...,Finalisation,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2019-06-18T13:24:20Z,11120;,Education facilities and training;,111;,"Education, Level Unspecified;"
4,XM-DAC-5-7-90024987,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[GE, TJ, UZ]",NaN,[Reg. Süd-Zentral Asien (nur ODA-Länder)],The PASCH initiative globally conntects pupils...,Die PASCH-Initiative verbindet über Deutsch al...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2019-06-13T15:10:54Z,11120;,Education facilities and training;,111;,"Education, Level Unspecified;"


### Documents

In [26]:
#df.document_link_url.value_counts()

try:
    trans_df['docs'] = df['document_link_url']
except:
    trans_df["docs"] = "NaN"
trans_df.head(2)

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798],[Reg. Asien (alle Länder)],strengthen national capacities to effectively ...,Förderung des Dialogs zu Themen des konvention...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-18T08:55:11Z,15240;,Reintegration and SALW control;,152;,"Conflict, Peace & Security;",NaN
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN,"[Rangun, Myanmar]",The people of Myanmar have improved knowledge ...,Die Bevölkerung von Myanmar weiß besser über d...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-17T21:33:37Z,15152;,Legislatures and political parties;,151;,Government & Civil Society-general;,NaN


## **Save final DF as CSV**

In [27]:
trans_df.head()

,iati_id,title_en,title_other,organization,country,region,location,description_en,description_other,status,planned_start,actual_start,planned_end,actual_end,last_update,crs_5_code,crs_5_name,crs_3_code,crs_3_name,docs
0,XM-DAC-5-7-6612417,Building Capacity of States in Southeast Asia ...,Durchführung Seminare zu Kleinwaffenkontrolle ...,Germany - Federal Foreign Office,NaN,[798],[Reg. Asien (alle Länder)],strengthen national capacities to effectively ...,Förderung des Dialogs zu Themen des konvention...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-18T08:55:11Z,15240;,Reintegration and SALW control;,152;,"Conflict, Peace & Security;",NaN
1,XM-DAC-5-7-6612437,Supporting a people-oriented Parliament in Mya...,Förderung eines bürgernahen Parlaments in Myan...,Germany - Federal Foreign Office,[MM],NaN,"[Rangun, Myanmar]",The people of Myanmar have improved knowledge ...,Die Bevölkerung von Myanmar weiß besser über d...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-17T21:33:37Z,15152;,Legislatures and political parties;,151;,Government & Civil Society-general;,NaN
2,XM-DAC-5-7-6612446,German Participation at National Science and T...,Deutsche Beteiligung an der National Science a...,Germany - Federal Foreign Office,[TH],NaN,"[Bangkok, Thailand]",On the basis of descriptive examples at the bo...,Anhand von anschaulichen Beispielen auf dem Me...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2021-08-18T08:42:47Z,41081;,Environmental education/training;,410;,General Environment Protection;,NaN
3,XM-DAC-5-7-90025547,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[CN, VN]",NaN,[Reg. Östl. Asien (nur ODA-Länder)],The PASCH initiative globally conntects pupils...,Die PASCH-Initiative verbindet über Deutsch al...,Finalisation,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2019-06-18T13:24:20Z,11120;,Education facilities and training;,111;,"Education, Level Unspecified;",NaN
4,XM-DAC-5-7-90024987,10 years initiative Schools - Partners for the...,10 Jahre Initiative Schulen - Partner der Zuku...,Germany - Federal Foreign Office,"[GE, TJ, UZ]",NaN,[Reg. Süd-Zentral Asien (nur ODA-Länder)],The PASCH initiative globally conntects pupils...,Die PASCH-Initiative verbindet über Deutsch al...,Closed,2023-01-01T00:00:00Z,NaN,2023-12-31T00:00:00Z,NaN,2019-06-13T15:10:54Z,11120;,Education facilities and training;,111;,"Education, Level Unspecified;",NaN


In [28]:
trans_df.to_csv(output_file, index=False) 